In [1]:
from snowflake.snowpark import Session
import warnings; warnings.simplefilter('ignore')
import configparser

In [2]:
# Fosfor ML
from fosforml import *
from fosforml.constants import MLModelFlavours
import requests

In [3]:
import pandas as pd
import numpy as np

# Import label encoder 
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt



In [4]:
#Import all snowflake connection details from template variables.
db_user = 'ASHUTOSH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'RETAIL_DB'
db_role = 'ASHUTOSH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'CPG_SAP_OTIF_SCHEMA'

In [5]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [6]:
df = session1.table("ENRICHED_ORDER_SHIP")

In [7]:
df = df.to_pandas()

In [8]:
df.shape

(966997, 46)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 46 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   CUSTOMER_PURCHASE_ORDER_DATE          966997 non-null  object 
 1   ORDER_ENTRY_DATE                      966997 non-null  object 
 2   ORDER_REQUIRED_DELIVERY_DATE          966997 non-null  object 
 3   ORDER_CUSTOMER_REQUEST_DELIVERY_DATE  966997 non-null  object 
 4   ORDER_PROMISE_DATE                    966997 non-null  object 
 5   ORDER_FIRST_SHIPMENT_DATE             966997 non-null  object 
 6   SHIPMENT_ACTUAL_START_DATE            966997 non-null  object 
 7   ORDER_LINECANCEL_DATE                 966997 non-null  object 
 8   ITEM_OTQF                             758063 non-null  object 
 9   ITEM_OTQF_FLAG                        966997 non-null  object 
 10  ITEM_OTQF_DAYS                        847138 non-null  float64
 11  

In [10]:
#Dropping all the ID related columns

cols_to_drop = ['ITEM_OTQF','ITEM_ORDERFILL','ITEM_CUSTOMER','ORDER_NUMBER']
df.drop(cols_to_drop, inplace=True, axis=1)

In [11]:
df.shape

(966997, 42)

In [12]:
cat_columns = ['CUSTOMER_NAME','SHIPMENT_ORIGINATION_FACILITY','PRODUCT_TYPE','PACKAGE_TYPE','PRODUCT_TECHNOLOGY',
'PRODUCT_SUBTECHNOLOGY','PRODUCT_DESIGN_TYPE','PRODUCT_SIZE','SUB_BRAND','CITY','OUTLETS','REGION','COUNTRY','CHANNEL',
'SHIPMENT_ORIGINATION_FACILITY_TYPE','BRAND']

for col in cat_columns:
    print ("------------------------------------------------")
    print (col, " -> ",df[col].nunique())

------------------------------------------------
CUSTOMER_NAME  ->  80
------------------------------------------------
SHIPMENT_ORIGINATION_FACILITY  ->  17
------------------------------------------------
PRODUCT_TYPE  ->  1
------------------------------------------------
PACKAGE_TYPE  ->  3
------------------------------------------------
PRODUCT_TECHNOLOGY  ->  1
------------------------------------------------
PRODUCT_SUBTECHNOLOGY  ->  3
------------------------------------------------
PRODUCT_DESIGN_TYPE  ->  3
------------------------------------------------
PRODUCT_SIZE  ->  7
------------------------------------------------
SUB_BRAND  ->  32
------------------------------------------------
CITY  ->  3369
------------------------------------------------
OUTLETS  ->  7
------------------------------------------------
REGION  ->  67
------------------------------------------------
COUNTRY  ->  6
------------------------------------------------
CHANNEL  ->  3
-------------------

In [13]:
df[cat_columns].head()

,CUSTOMER_NAME,SHIPMENT_ORIGINATION_FACILITY,PRODUCT_TYPE,PACKAGE_TYPE,PRODUCT_TECHNOLOGY,PRODUCT_SUBTECHNOLOGY,PRODUCT_DESIGN_TYPE,PRODUCT_SIZE,SUB_BRAND,CITY,OUTLETS,REGION,COUNTRY,CHANNEL,SHIPMENT_ORIGINATION_FACILITY_TYPE,BRAND
0,WALMART - US,Yuma Plant,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,El Paso,Hyperstore,TX,United States,Modern Channel,Distribution center,Craisins Trail
1,WALMART - US,Fort Wayne DC,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Somerset,Hyperstore,KY,United States,Modern Channel,Distribution center,Craisins Trail
2,WALMART - US,Fort Wayne DC,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Ludington,Hyperstore,MI,United States,Modern Channel,Distribution center,Craisins Trail
3,WALMART - US,San Antonio 3PL,Beverage,Regular,Soft Drinks,Energy Drink,Cans,16 Oz,Craisins Trail,La Grange,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail
4,WALMART - US,San Antonio 3PL,Beverage,Regular,Soft Drinks,Aerated Drinks,Cans,12 Oz,Craisins Trail,Uvalde,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail


In [14]:
# Drop categorical columns
cat_cols_to_drop = ['PRODUCT_TYPE','PRODUCT_TECHNOLOGY','CITY','SUB_BRAND']
df.drop(cat_cols_to_drop, inplace=True, axis=1)

In [15]:
df.shape

(966997, 38)

In [16]:
date_columns = [
'CUSTOMER_PURCHASE_ORDER_DATE','ORDER_ENTRY_DATE','ORDER_REQUIRED_DELIVERY_DATE','ORDER_CUSTOMER_REQUEST_DELIVERY_DATE',
    'ORDER_PROMISE_DATE','ORDER_FIRST_SHIPMENT_DATE','SHIPMENT_ACTUAL_START_DATE','ORDER_LINECANCEL_DATE'
]

In [17]:
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

In [18]:
df['CUSTOMER_ORDER_YEAR'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.year
df['CUSTOMER_ORDER_MONTH'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.month
df['CUSTOMER_ORDER_WEEK'] = df['CUSTOMER_PURCHASE_ORDER_DATE'].dt.weekday

In [19]:
df['CUSTOMER_ORDER_ENTRY_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_ENTRY_DATE']).dt.days
df['CUSTOMER_ORDER_REQUIRE_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_REQUIRED_DELIVERY_DATE']).dt.days
df['CUSTOMER_ORDER_REQ_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_CUSTOMER_REQUEST_DELIVERY_DATE']).dt.days
df['CUSTOMER_ORDER_PROM_DELV_IN_DAYS'] = (df['CUSTOMER_PURCHASE_ORDER_DATE'] - df['ORDER_PROMISE_DATE']).dt.days

df['ORDER_ENTRY_PROMISE_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['ORDER_PROMISE_DATE']).dt.days
df['ORDER_ENTRY_SHIPMT_START_IN_DAYS'] = (df['ORDER_ENTRY_DATE'] - df['SHIPMENT_ACTUAL_START_DATE']).dt.days

In [20]:
df.drop(date_columns, inplace=True, axis=1)

In [21]:
df.shape

(966997, 39)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 39 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ITEM_OTQF_FLAG                       966997 non-null  object 
 1   ITEM_OTQF_DAYS                       847138 non-null  float64
 2   ITEM_ORDERFILL_FLAG                  966997 non-null  object 
 3   ITEM_ORDERFILL_DAYS                  957859 non-null  float64
 4   ITEN_CUSTOMER_ORDERFILL_FLAG         966997 non-null  int8   
 5   ITEM_CUSTOER_DAYS                    847138 non-null  float64
 6   OPEN_ORDER_FLAG                      966997 non-null  object 
 7   BACK_ORDER_FLAG                      966997 non-null  object 
 8   CANCEL_FLAG                          966997 non-null  object 
 9   CUSTOMER_NAME                        966997 non-null  object 
 10  SHIPMENT_ORIGINATION_FACILITY        966997 non-null  object 
 11  PACKAGE_TYPE 

In [23]:
df.fillna(0, inplace=True)

In [24]:
df.isnull().sum()

ITEM_OTQF_FLAG                         0
ITEM_OTQF_DAYS                         0
ITEM_ORDERFILL_FLAG                    0
ITEM_ORDERFILL_DAYS                    0
ITEN_CUSTOMER_ORDERFILL_FLAG           0
ITEM_CUSTOER_DAYS                      0
OPEN_ORDER_FLAG                        0
BACK_ORDER_FLAG                        0
CANCEL_FLAG                            0
CUSTOMER_NAME                          0
SHIPMENT_ORIGINATION_FACILITY          0
PACKAGE_TYPE                           0
PRODUCT_SUBTECHNOLOGY                  0
PRODUCT_DESIGN_TYPE                    0
PRODUCT_SIZE                           0
OUTLETS                                0
ORDER_QUANTITY                         0
QUANTITY_SHIPPED_ORDERFILL             0
QUANTITY_SHIPPED_CUSTOMER              0
QUANTITY_SHIPPED_OTQF                  0
DELIVERED_QUANTITY                     0
DELIVERED_QUANTITY_OTQF                0
OPEN_QUANTITY                          0
BACK_ORDER_QUANTITY                    0
CANCEL_QUANTITY 

In [26]:
df['ITEM_OTQF_FLAG'].value_counts()

ITEM_OTQF_FLAG
1    758063
0    208934
Name: count, dtype: int64

In [27]:
df['ITEM_ORDERFILL_FLAG'].value_counts()

ITEM_ORDERFILL_FLAG
1    856681
0    110316
Name: count, dtype: int64

In [28]:
df['OPEN_ORDER_FLAG'].value_counts()

OPEN_ORDER_FLAG
0    966313
1       684
Name: count, dtype: int64

In [56]:
df['BACK_ORDER_FLAG'].value_counts()

BACK_ORDER_FLAG
0    966765
1       232
Name: count, dtype: int64

In [29]:
df['CANCEL_FLAG'].value_counts()

CANCEL_FLAG
0    877244
1     89753
Name: count, dtype: int64

In [30]:
df.drop('CANCEL_FLAG', inplace=True, axis=1)

In [31]:
for col in ['ITEM_OTQF_FLAG','ITEM_ORDERFILL_FLAG','OPEN_ORDER_FLAG','BACK_ORDER_FLAG']:
    df[col] = df[col].astype('int')

In [32]:
df.select_dtypes(include='object')

,CUSTOMER_NAME,SHIPMENT_ORIGINATION_FACILITY,PACKAGE_TYPE,PRODUCT_SUBTECHNOLOGY,PRODUCT_DESIGN_TYPE,PRODUCT_SIZE,OUTLETS,REGION,COUNTRY,CHANNEL,SHIPMENT_ORIGINATION_FACILITY_TYPE,BRAND
0,WALMART - US,Yuma Plant,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,TX,United States,Modern Channel,Distribution center,Craisins Trail
1,WALMART - US,Fort Wayne DC,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,KY,United States,Modern Channel,Distribution center,Craisins Trail
2,WALMART - US,Fort Wayne DC,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,MI,United States,Modern Channel,Distribution center,Craisins Trail
3,WALMART - US,San Antonio 3PL,Regular,Energy Drink,Cans,16 Oz,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail
4,WALMART - US,San Antonio 3PL,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,TX,United States,Modern Channel,3PL Third Party Logistics,Craisins Trail
...,...,...,...,...,...,...,...,...,...,...,...,...
966992,INTERSTATE BATTERIES US,San Antonio 3PL,Regular,Aerated Drinks,Cans,24 Oz,SuperStore,MS,United States,Modern Channel,3PL Third Party Logistics,Rock star Energy
966993,WALMART - US,Portland Plant,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,CA,United States,Modern Channel,Fully Integrated Plant,Maxwell Home
966994,WALMART - US,Portland Plant,Regular,Aerated Drinks,Cans,22 Oz,Hyperstore,OR,United States,Modern Channel,Fully Integrated Plant,Craisins Trail
966995,WALMART - US,Portland Plant,Regular,Aerated Drinks,Cans,12 Oz,Hyperstore,WA,United States,Modern Channel,Fully Integrated Plant,Maxwell Home


In [33]:
df.shape

(966997, 38)

In [36]:
df.select_dtypes(exclude='object').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 26 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ITEM_OTQF_FLAG                       966997 non-null  int64  
 1   ITEM_OTQF_DAYS                       966997 non-null  float64
 2   ITEM_ORDERFILL_FLAG                  966997 non-null  int64  
 3   ITEM_ORDERFILL_DAYS                  966997 non-null  float64
 4   ITEN_CUSTOMER_ORDERFILL_FLAG         966997 non-null  int8   
 5   ITEM_CUSTOER_DAYS                    966997 non-null  float64
 6   OPEN_ORDER_FLAG                      966997 non-null  int64  
 7   BACK_ORDER_FLAG                      966997 non-null  int64  
 8   ORDER_QUANTITY                       966997 non-null  int16  
 9   QUANTITY_SHIPPED_ORDERFILL           966997 non-null  int16  
 10  QUANTITY_SHIPPED_CUSTOMER            966997 non-null  int16  
 11  QUANTITY_SHIP

In [37]:
num_to_drop = ['DELIVERED_QUANTITY','CANCEL_QUANTITY','QUANTITY_SHIPPED_ORDERFILL','QUANTITY_SHIPPED_CUSTOMER','QUANTITY_SHIPPED_OTQF']
df.drop(num_to_drop, inplace=True, axis=1)

In [38]:
df.shape

(966997, 33)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966997 entries, 0 to 966996
Data columns (total 33 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ITEM_OTQF_FLAG                       966997 non-null  int64  
 1   ITEM_OTQF_DAYS                       966997 non-null  float64
 2   ITEM_ORDERFILL_FLAG                  966997 non-null  int64  
 3   ITEM_ORDERFILL_DAYS                  966997 non-null  float64
 4   ITEN_CUSTOMER_ORDERFILL_FLAG         966997 non-null  int8   
 5   ITEM_CUSTOER_DAYS                    966997 non-null  float64
 6   OPEN_ORDER_FLAG                      966997 non-null  int64  
 7   BACK_ORDER_FLAG                      966997 non-null  int64  
 8   CUSTOMER_NAME                        966997 non-null  object 
 9   SHIPMENT_ORIGINATION_FACILITY        966997 non-null  object 
 10  PACKAGE_TYPE                         966997 non-null  object 
 11  PRODUCT_SUBTE